In [ ]:
# 해당 모든 코드는 Google Colab을 통해서 실행되었음을 알려드립니다.

In [ ]:
import sys
print(sys.version)

3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]


In [ ]:
!pip install catboost==1.2.8
!pip install optuna==4.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.9/583.9 kB 30.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import catboost
import sklearn
import optuna

versions = {
    "pandas": pd.__version__,
    "numpy": np.__version__,
    "catboost": catboost.__version__,
    "sklearn": sklearn.__version__,
    "optuna": optuna.__version__,
}

import pprint
pprint.pprint(versions)

{'catboost': '1.2.8',
 'numpy': '2.0.2',
 'optuna': '4.3.0',
 'pandas': '2.2.2',
 'sklearn': '1.6.1'}


In [ ]:
pip freeze > requirements.txt

### Organize the files

In [ ]:
# 파일 정리에는 v2-8 TPU을 사용.

In [ ]:
import pandas as pd
from google.colab import drive
import os

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_folder = '/content/drive/MyDrive/base_file/'

months = ["07", "08", "09", "10", "11", "12"]

categories = ["회원정보", "신용정보", "승인매출정보", "청구입금정보", "잔액정보", "채널정보", "마케팅정보", "성과정보"]
data_types = ["train", "test"]

def merge_monthly_data(data_type, category):
    merged_list = []

    for month in months:
        file_name = f"{drive_folder}2018{month}_{data_type}_{category}.parquet"
        try:
            df = pd.read_parquet(file_name, engine="pyarrow")
            merged_list.append(df)
            print(f"✅ {file_name} 변환 완료")
        except FileNotFoundError:
            print(f"⚠️ 파일 없음: {file_name}")

    if merged_list:
        merged_df = pd.concat(merged_list, ignore_index=True)
        output_file = f"{drive_folder}{data_type}_{category}.csv"
        merged_df.to_csv(output_file, index=False)
        print(f"✅ {output_file} 저장 완료 (Shape: {merged_df.shape})")
    else:
        print(f"❌ {data_type}_{category} 데이터 없음")

for data_type in data_types:
    for category in categories:
        merge_monthly_data(data_type, category)

✅ /content/drive/MyDrive/base_file/201807_train_회원정보.parquet 변환 완료
✅ /content/drive/MyDrive/base_file/201808_train_회원정보.parquet 변환 완료
✅ /content/drive/MyDrive/base_file/201809_train_회원정보.parquet 변환 완료
✅ /content/drive/MyDrive/base_file/201810_train_회원정보.parquet 변환 완료
✅ /content/drive/MyDrive/base_file/201811_train_회원정보.parquet 변환 완료
✅ /content/drive/MyDrive/base_file/201812_train_회원정보.parquet 변환 완료
✅ /content/drive/MyDrive/base_file/train_회원정보.csv 저장 완료 (Shape: (2400000, 78))
✅ /content/drive/MyDrive/base_file/201807_train_신용정보.parquet 변환 완료
✅ /content/drive/MyDrive/base_file/201808_train_신용정보.parquet 변환 완료
✅ /content/drive/MyDrive/base_file/201809_train_신용정보.parquet 변환 완료
✅ /content/drive/MyDrive/base_file/201810_train_신용정보.parquet 변환 완료
✅ /content/drive/MyDrive/base_file/201811_train_신용정보.parquet 변환 완료
✅ /content/drive/MyDrive/base_file/201812_train_신용정보.parquet 변환 완료
✅ /content/drive/MyDrive/base_file/train_신용정보.csv 저장 완료 (Shape: (2400000, 42))
✅ /content/drive/MyDrive/base_file/201

In [ ]:
base_path = "/content/drive/MyDrive/base_file/"

file_names = [
    "train_회원정보.csv",
    "train_신용정보.csv",
    "train_승인매출정보.csv",
    "train_청구입금정보.csv",
    "train_잔액정보.csv",
    "train_채널정보.csv",
    "train_마케팅정보.csv",
    "train_성과정보.csv"
]

df = pd.read_csv(base_path + file_names[0])

for idx, file in enumerate(file_names[1:], start=2):
    print(f"\n🔹 병합 중: {file} ({idx}/{len(file_names)})")
    temp_df = pd.read_csv(base_path + file)

    df = df.merge(temp_df, how="left", on=["ID", "기준년월"])
    print(f"✅ 병합 후 크기: {df.shape}")

output_file = base_path + "base_train.csv"
df.to_csv(output_file, index=False)

print(f"\n✅ 최종 데이터 저장 완료: {output_file}")
print(f"🧾 최종 데이터 크기: {df.shape[0]}행, {df.shape[1]}열")


🔹 병합 중: train_신용정보.csv (2/8)
✅ 병합 후 크기: (2400000, 118)

🔹 병합 중: train_승인매출정보.csv (3/8)


<ipython-input-9-208cf7ea5a6c>:18: DtypeWarning: Columns (183) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(base_path + file)


✅ 병합 후 크기: (2400000, 522)

🔹 병합 중: train_청구입금정보.csv (4/8)
✅ 병합 후 크기: (2400000, 566)

🔹 병합 중: train_잔액정보.csv (5/8)
✅ 병합 후 크기: (2400000, 646)

🔹 병합 중: train_채널정보.csv (6/8)
✅ 병합 후 크기: (2400000, 749)

🔹 병합 중: train_마케팅정보.csv (7/8)
✅ 병합 후 크기: (2400000, 811)

🔹 병합 중: train_성과정보.csv (8/8)
✅ 병합 후 크기: (2400000, 858)

✅ 최종 데이터 저장 완료: /content/drive/MyDrive/base_file/base_train.csv
🧾 최종 데이터 크기: 2400000행, 858열


In [ ]:
base_path = "/content/drive/MyDrive/base_file/"

file_names = [
    "test_회원정보.csv",
    "test_신용정보.csv",
    "test_승인매출정보.csv",
    "test_청구입금정보.csv",
    "test_잔액정보.csv",
    "test_채널정보.csv",
    "test_마케팅정보.csv",
    "test_성과정보.csv"
]

df = pd.read_csv(base_path + file_names[0])

for idx, file in enumerate(file_names[1:], start=2):
    print(f"\n🔹 병합 중: {file} ({idx}/{len(file_names)})")
    temp_df = pd.read_csv(base_path + file)

    df = df.merge(temp_df, how="left", on=["ID", "기준년월"])
    print(f"✅ 병합 후 크기: {df.shape}")

output_file = base_path + "base_test.csv"
df.to_csv(output_file, index=False)

print(f"\n✅ 최종 데이터 저장 완료: {output_file}")
print(f"🧾 최종 데이터 크기: {df.shape[0]}행, {df.shape[1]}열")


🔹 병합 중: test_신용정보.csv (2/8)
✅ 병합 후 크기: (600000, 117)

🔹 병합 중: test_승인매출정보.csv (3/8)


<ipython-input-10-e90a1969e516>:18: DtypeWarning: Columns (183) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(base_path + file)


✅ 병합 후 크기: (600000, 521)

🔹 병합 중: test_청구입금정보.csv (4/8)
✅ 병합 후 크기: (600000, 565)

🔹 병합 중: test_잔액정보.csv (5/8)
✅ 병합 후 크기: (600000, 645)

🔹 병합 중: test_채널정보.csv (6/8)
✅ 병합 후 크기: (600000, 748)

🔹 병합 중: test_마케팅정보.csv (7/8)
✅ 병합 후 크기: (600000, 810)

🔹 병합 중: test_성과정보.csv (8/8)
✅ 병합 후 크기: (600000, 857)

✅ 최종 데이터 저장 완료: /content/drive/MyDrive/base_file/base_test.csv
🧾 최종 데이터 크기: 600000행, 857열


In [ ]:
base_path = "/content/drive/MyDrive/base_file/"

file_names = [
    "train_회원정보.csv",
    "train_신용정보.csv",
    "train_승인매출정보.csv",
    "train_청구입금정보.csv",
    "train_잔액정보.csv",
    "train_채널정보.csv",
    "train_마케팅정보.csv",
    "train_성과정보.csv"
]

df = pd.read_csv(base_path + file_names[0])

original_shape = df.shape

for idx, file in enumerate(file_names[1:], start=2):
    print(f"\n🔹 병합 진행 중: {file} (파일 {idx} / {len(file_names)})")

    temp_df = pd.read_csv(base_path + file)
    df = df.merge(temp_df, how="left", on=["ID", "기준년월"])
    print(f"✅ 병합 후 데이터 크기: {df.shape[0]}행, {df.shape[1]}열")

    constant_cols = [col for col in df.columns if df[col].nunique() == 1]
    if constant_cols:
        print(f"📌 제거된 모든 값이 동일한 칼럼: {constant_cols}")
        df = df.drop(columns=constant_cols)
    else:
        print("📌 모든 값이 동일한 칼럼 없음")

    col_groups = {}
    for col in df.columns:
        for key in col_groups:
            if df[col].equals(df[key]):
                col_groups[key].append(col)
                break
        else:
            col_groups[col] = [col]

    duplicate_cols = [col for group in col_groups.values() for col in group[1:]]
    if duplicate_cols:
        print(f"📌 제거된 중복 칼럼: {duplicate_cols}")
        df = df.drop(columns=duplicate_cols)
    else:
        print("📌 중복 칼럼 없음")

    if 'ID' in df.columns and df.columns.str.contains('ID').sum() > 1:
        df = df.loc[:, ~df.columns.str.contains('ID', case=False)].join(df[['ID']])

    if '기준년월' in df.columns and df.columns.str.contains('기준년월').sum() > 1:
        df = df.loc[:, ~df.columns.str.contains('기준년월', case=False)].join(df[['기준년월']])

    print(f"🔹 {file} 처리 완료. 현재 데이터 크기: {df.shape[0]}행, {df.shape[1]}열")

new_shape = df.shape

output_file = base_path + "base_clean_train.csv"
df.to_csv(output_file, index=False)

print(f"\n✅ 원래 데이터 크기: {original_shape[0]}행, {original_shape[1]}열")
print(f"✅ 병합 후 최종 데이터 크기: {new_shape[0]}행, {new_shape[1]}열")

print(f"\n✅ 최종 데이터 저장 완료: {output_file}")


🔹 병합 진행 중: train_신용정보.csv (파일 2 / 8)
✅ 병합 후 데이터 크기: 2400000행, 118열
📌 제거된 모든 값이 동일한 칼럼: ['이용카드수_체크_가족', '이용금액_R3M_체크_가족', '연회비할인카드수_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M', '상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M', '시장연체상환여부_R3M']
📌 제거된 중복 칼럼: ['청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M']
🔹 train_신용정보.csv 처리 완료. 현재 데이터 크기: 2400000행, 106열

🔹 병합 진행 중: train_승인매출정보.csv (파일 3 / 8)


<ipython-input-11-b4b3d50a070f>:21: DtypeWarning: Columns (183) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(base_path + file)


✅ 병합 후 데이터 크기: 2400000행, 510열
📌 제거된 모든 값이 동일한 칼럼: ['이용건수_부분무이자_B0M', '이용금액_부분무이자_B0M', '여유_여행이용금액', '납부_렌탈료이용금액', '납부_유선방송이용금액', '납부_건강연금이용금액', '할부건수_부분_3M_R12M', '할부건수_부분_6M_R12M', '할부건수_부분_14M_R12M', '할부금액_부분_3M_R12M', 'RP건수_유선방송_B0M', 'RP건수_건강_B0M', 'RP후경과월_유선방송', 'RP후경과월_건강', '증감_RP건수_유선방송_전월', '증감_RP건수_건강_전월', '이용개월수_당사페이_R6M', '이용금액_당사페이_R6M', '이용금액_당사기타_R6M', '이용건수_당사페이_R6M', '이용건수_당사기타_R6M', '이용금액_당사페이_R3M', '이용금액_당사기타_R3M', '이용건수_당사페이_R3M', '이용건수_당사기타_R3M', '이용금액_당사페이_B0M', '이용금액_당사기타_B0M', '이용건수_당사페이_B0M', '이용건수_당사기타_B0M', '승인거절건수_입력오류_B0M', '승인거절건수_기타_B0M']
📌 제거된 중복 칼럼: ['이용횟수_연체_B0M', '할부건수_부분_12M_R12M']
🔹 train_승인매출정보.csv 처리 완료. 현재 데이터 크기: 2400000행, 477열

🔹 병합 진행 중: train_청구입금정보.csv (파일 4 / 8)
✅ 병합 후 데이터 크기: 2400000행, 521열
📌 제거된 모든 값이 동일한 칼럼: ['대표결제방법코드']
📌 중복 칼럼 없음
🔹 train_청구입금정보.csv 처리 완료. 현재 데이터 크기: 2400000행, 520열

🔹 병합 진행 중: train_잔액정보.csv (파일 5 / 8)
✅ 병합 후 데이터 크기: 2400000행, 600열
📌 제거된 모든 값이 동일한 칼럼: ['카드론잔액_최종경과월', '최종연체개월수_R15M', 'RV잔액이월횟수_R6M', 'RV잔액이월횟수_R3M', '연체잔액_

In [ ]:
base_path = "/content/drive/MyDrive/base_file/"

test_file_names = [
    "test_회원정보.csv",
    "test_신용정보.csv",
    "test_승인매출정보.csv",
    "test_청구입금정보.csv",
    "test_잔액정보.csv",
    "test_채널정보.csv",
    "test_마케팅정보.csv",
    "test_성과정보.csv"
]

test_df = pd.read_csv(base_path + test_file_names[0])

test_original_shape = test_df.shape
for idx, file in enumerate(test_file_names[1:], start=2):
    print(f"\n🔹 병합 진행 중: {file} (파일 {idx} / {len(test_file_names)})")
    temp_df = pd.read_csv(base_path + file)
    test_df = test_df.merge(temp_df, how="left", on=["ID", "기준년월"])
    print(f"✅ 병합 후 데이터 크기: {test_df.shape[0]}행, {test_df.shape[1]}열")

train_df = pd.read_csv(base_path + "base_clean_train.csv", nrows=1)
train_columns = train_df.columns
test_columns_to_keep = [col for col in test_df.columns if col in train_columns]

test_df = test_df[test_columns_to_keep]
test_final_shape = test_df.shape

test_output_file = base_path + "base_clean_test.csv"
test_df.to_csv(test_output_file, index=False)

print(f"\n✅ 원래 test 데이터 크기: {test_original_shape[0]}행, {test_original_shape[1]}열")
print(f"✅ 병합 후 최종 test 데이터 크기: {test_final_shape[0]}행, {test_final_shape[1]}열")
print(f"\n✅ 최종 test 데이터 저장 완료: {test_output_file}")

train_col_set = set(train_columns)
test_col_set = set(test_df.columns)

if train_col_set == test_col_set:
    print("\n✅ train과 test의 컬럼이 완전히 일치합니다!")
else:
    train_only_cols = train_col_set - test_col_set
    test_only_cols = test_col_set - train_col_set

    print(f"\n⚠️ train과 test의 컬럼이 다릅니다!")
    print(f"🔹 train에만 있는 컬럼 ({len(train_only_cols)}개): {train_only_cols}")
    print(f"🔹 test에만 있는 컬럼 ({len(test_only_cols)}개): {test_only_cols}")


🔹 병합 진행 중: test_신용정보.csv (파일 2 / 8)
✅ 병합 후 데이터 크기: 600000행, 117열

🔹 병합 진행 중: test_승인매출정보.csv (파일 3 / 8)


<ipython-input-12-2e073f64f627>:19: DtypeWarning: Columns (183) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(base_path + file)


✅ 병합 후 데이터 크기: 600000행, 521열

🔹 병합 진행 중: test_청구입금정보.csv (파일 4 / 8)
✅ 병합 후 데이터 크기: 600000행, 565열

🔹 병합 진행 중: test_잔액정보.csv (파일 5 / 8)
✅ 병합 후 데이터 크기: 600000행, 645열

🔹 병합 진행 중: test_채널정보.csv (파일 6 / 8)
✅ 병합 후 데이터 크기: 600000행, 748열

🔹 병합 진행 중: test_마케팅정보.csv (파일 7 / 8)
✅ 병합 후 데이터 크기: 600000행, 810열

🔹 병합 진행 중: test_성과정보.csv (파일 8 / 8)
✅ 병합 후 데이터 크기: 600000행, 857열

✅ 원래 test 데이터 크기: 600000행, 77열
✅ 병합 후 최종 test 데이터 크기: 600000행, 745열

✅ 최종 test 데이터 저장 완료: /content/drive/MyDrive/base_file/base_clean_test.csv

⚠️ train과 test의 컬럼이 다릅니다!
🔹 train에만 있는 컬럼 (1개): {'Segment'}
🔹 test에만 있는 컬럼 (0개): set()


### BASE model

In [ ]:
# BASE model에는 A100 GPU을 사용.

In [ ]:
!pip install catboost==1.2.8
!pip install optuna==4.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 23.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import optuna
import json

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/base_file/base_clean_train.csv')
test = pd.read_csv('/content/drive/MyDrive/base_file/base_clean_test.csv')

ab_ids = train[train['Segment'].isin(['A', 'B'])]['ID'].unique()

train = train[~train['ID'].isin(ab_ids)].copy()

<ipython-input-5-e7b1426f6e19>:1: DtypeWarning: Columns (281,355) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('/content/drive/MyDrive/base_file/base_clean_train.csv')
<ipython-input-5-e7b1426f6e19>:2: DtypeWarning: Columns (280) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('/content/drive/MyDrive/base_file/base_clean_test.csv')


In [ ]:
# label_encoder = LabelEncoder()
# train['Segment'] = label_encoder.fit_transform(train['Segment'])

# X = train.drop(columns=['Segment', 'ID'])
# y = train['Segment']
# X_test = test.drop(columns=['ID'])

# cat_features = [col for col in X.columns if X[col].dtype == 'object']
# for col in cat_features:
#     X[col] = X[col].astype(str)
#     X_test[col] = X_test[col].astype(str)

# def objective(trial):
#     bootstrap_type = trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli'])

#     params = {
#         'iterations': 1500,
#         'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.3),
#         'depth': 8,
#         'l2_leaf_reg': trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
#         'random_strength': trial.suggest_float("random_strength", 1e-9, 10.0),
#         'border_count': trial.suggest_int("border_count", 32, 255),
#         'bootstrap_type': bootstrap_type,
#         'task_type': 'GPU',
#         'loss_function': 'MultiClass',
#         'eval_metric': 'TotalF1',
#         'verbose': 0,
#         'random_seed': 42,
#         'class_weights': [2, 1, 1]
#     }

#     if bootstrap_type == 'Bayesian':
#         params['bagging_temperature'] = trial.suggest_float("bagging_temperature", 0.0, 1.0)
#     else:
#         params['subsample'] = trial.suggest_float("subsample", 0.5, 1.0)

#     X_train_sub, X_valid, y_train_sub, y_valid = train_test_split(
#         X, y, test_size=0.2, stratify=y, random_state=42
#     )

#     model = CatBoostClassifier(**params)
#     model.fit(
#         X_train_sub, y_train_sub,
#         eval_set=(X_valid, y_valid),
#         cat_features=cat_features,
#         use_best_model=True,
#         early_stopping_rounds=100
#     )

#     preds = model.predict(X_valid)
#     score = f1_score(y_valid, preds, average='macro')
#     return score

# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=30)

# best_params = study.best_trial.params
# with open('/content/drive/MyDrive/base_file/base_catboost_params.json', 'w') as f:
#     json.dump(best_params, f)

# best_params.update({
#     'iterations': 1500,
#     'loss_function': 'MultiClass',
#     'eval_metric': 'TotalF1',
#     'task_type': 'GPU',
#     'verbose': 100,
#     'random_seed': 42,
#     'depth': 8,
#     'class_weights': [2, 1, 1]
# })

# final_model = CatBoostClassifier(**best_params)
# final_model.fit(X, y, cat_features=cat_features)

# probs = final_model.predict_proba(X_test)
# prob_df = pd.DataFrame(probs, columns=range(probs.shape[1]))
# prob_df['ID'] = test['ID'].values

# mean_probs = prob_df.groupby('ID').mean().reset_index()
# mean_probs['Segment'] = mean_probs.drop(columns='ID').values.argmax(axis=1)

# segment_mapping = {0: 'C', 1: 'D', 2: 'E'}
# mean_probs['Segment'] = mean_probs['Segment'].map(segment_mapping)

# submission = pd.DataFrame({'ID': mean_probs['ID'], 'Segment': mean_probs['Segment']})
# submission.to_csv('/content/drive/MyDrive/base_file/base_catboost.csv', index=False)

# print("✅ CatBoost + Optuna (단일 valid split) 예측 완료 및 저장 🎯")

In [ ]:
label_encoder = LabelEncoder()
train['Segment'] = label_encoder.fit_transform(train['Segment'])

X = train.drop(columns=['Segment', 'ID'])
y = train['Segment']
X_test = test.drop(columns=['ID'])

cat_features = [col for col in X.columns if X[col].dtype == 'object']
for col in cat_features:
    X[col] = X[col].astype(str)
    X_test[col] = X_test[col].astype(str)

best_params = {
    "bootstrap_type": "Bayesian",
    "learning_rate": 0.2997682904093563,
    "l2_leaf_reg": 9.214022161348987,
    "random_strength": 7.342192789415524,
    "bagging_temperature": 0.11417356499443036,
    "border_count": 251,
    "iterations": 1500,
    "loss_function": "MultiClass",
    "eval_metric": "TotalF1",
    "task_type": "GPU",
    "verbose": 100,
    "random_seed": 42,
    "depth": 8,
    "class_weights": [2, 1, 1]
}

n_classes = len(np.unique(y))
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
all_test_probs = np.zeros((X_test.shape[0], n_classes))

for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
    print(f"🚀 Fold {fold+1} training...")

    X_train_fold, y_train_fold = X.iloc[train_idx], y.iloc[train_idx]
    X_valid_fold, y_valid_fold = X.iloc[valid_idx], y.iloc[valid_idx]

    model = CatBoostClassifier(**best_params)
    model.fit(X_train_fold, y_train_fold, cat_features=cat_features)

    fold_probs = model.predict_proba(X_test)
    all_test_probs += fold_probs

avg_test_probs = all_test_probs / kf.get_n_splits()
prob_df = pd.DataFrame(avg_test_probs, columns=range(n_classes))
prob_df['ID'] = test['ID'].values

mean_probs = prob_df.groupby('ID').mean().reset_index()
mean_probs['Segment'] = mean_probs.drop(columns='ID').values.argmax(axis=1)

segment_mapping = {0: 'C', 1: 'D', 2: 'E'}
mean_probs['Segment'] = mean_probs['Segment'].map(segment_mapping)

submission = pd.DataFrame({'ID': mean_probs['ID'], 'Segment': mean_probs['Segment']})
submission.to_csv('/content/drive/MyDrive/base_file/base_catboost_kfold.csv', index=False)

print("✅ CatBoost + 10-Fold CV 예측 완료 및 저장 🎯")

🚀 Fold 1 training...
0:	learn: 0.8425386	total: 307ms	remaining: 7m 40s
100:	learn: 0.9042936	total: 6.49s	remaining: 1m 29s
200:	learn: 0.9178637	total: 12.4s	remaining: 1m 20s
300:	learn: 0.9270462	total: 18.5s	remaining: 1m 13s
400:	learn: 0.9341691	total: 24.5s	remaining: 1m 7s
500:	learn: 0.9401282	total: 30.4s	remaining: 1m
600:	learn: 0.9450037	total: 36.5s	remaining: 54.6s
700:	learn: 0.9493407	total: 42.3s	remaining: 48.3s
800:	learn: 0.9530282	total: 48.2s	remaining: 42.1s
900:	learn: 0.9565139	total: 54.2s	remaining: 36s
1000:	learn: 0.9596573	total: 60s	remaining: 29.9s
1100:	learn: 0.9622827	total: 1m 5s	remaining: 23.9s
1200:	learn: 0.9649718	total: 1m 11s	remaining: 17.9s
1300:	learn: 0.9672248	total: 1m 17s	remaining: 11.9s
1400:	learn: 0.9692903	total: 1m 23s	remaining: 5.92s
1499:	learn: 0.9710659	total: 1m 29s	remaining: 0us
🚀 Fold 2 training...
0:	learn: 0.8426310	total: 79.1ms	remaining: 1m 58s
100:	learn: 0.9042342	total: 6.27s	remaining: 1m 26s
200:	learn: 0.9183

### VIP model

In [ ]:
# VIP model에는 재현성 유지를 위해 L4 GPU 사용.
# 파일 생성 부분 RAM 부족 문제로 v2-8 TPU 사용. L4 GPU 전환지점 따로 표시 예정.

In [ ]:
from google.colab import drive
import pandas as pd

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/base_file/base_clean_train.csv')
test = pd.read_csv('/content/drive/MyDrive/base_file/base_clean_test.csv')

train_A = train[train['Segment'] == 'A']

cols_to_check = [col for col in train.columns if col not in ['ID', 'Segment']]

def is_fixed_column(df, col):
    return df[col].nunique() == 1

fixed_columns_A = {col: train_A[col].iloc[0] for col in cols_to_check if is_fixed_column(train_A, col)}
max_column_values = fixed_columns_A.copy()

fixed_cols = list(max_column_values.keys())

print(f"📦 고정된 칼럼 {len(fixed_cols)}개 제거할 예정입니다.")

<ipython-input-3-0b829fc33286>:1: DtypeWarning: Columns (281,355) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('/content/drive/MyDrive/base_file/base_clean_train.csv')
<ipython-input-3-0b829fc33286>:2: DtypeWarning: Columns (280) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('/content/drive/MyDrive/base_file/base_clean_test.csv')


📦 고정된 칼럼 83개 제거할 예정입니다.


In [ ]:
matching_ids_train = train.copy()
for col, value in max_column_values.items():
    matching_ids_train = matching_ids_train[matching_ids_train[col] == value]

matching_ids_train_grouped = matching_ids_train.groupby('ID').filter(lambda x: len(x) == 6)
matching_ids_train_list = matching_ids_train_grouped['ID'].unique()

matching_ids_test = test.copy()
for col, value in max_column_values.items():
    matching_ids_test = matching_ids_test[matching_ids_test[col] == value]

matching_ids_test_grouped = matching_ids_test.groupby('ID').filter(lambda x: len(x) == 6)
matching_ids_test_list = matching_ids_test_grouped['ID'].unique()

train_filtered = train[train['ID'].isin(matching_ids_train_list)].drop(columns=fixed_cols)
test_filtered = test[test['ID'].isin(matching_ids_test_list)].drop(columns=fixed_cols)

print(f"🚀 최종 train 데이터 shape: {train_filtered.shape}")
print(f"🚀 최종 test 데이터 shape: {test_filtered.shape}")

train_filtered.to_csv('/content/drive/MyDrive/base_file/train_vips_A.csv', index=False)
test_filtered.to_csv('/content/drive/MyDrive/base_file/test_vips_A.csv', index=False)

🚀 최종 train 데이터 shape: (249594, 663)
🚀 최종 test 데이터 shape: (62586, 662)


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/base_file/base_clean_train.csv')
test = pd.read_csv('/content/drive/MyDrive/base_file/base_clean_test.csv')

train_B = train[train['Segment'] == 'B']

cols_to_check = [col for col in train.columns if col not in ['ID', 'Segment']]

def is_fixed_column(df, col):
    return df[col].nunique() == 1

fixed_columns_B = {col: train_B[col].iloc[0] for col in cols_to_check if is_fixed_column(train_B, col)}
max_column_values = fixed_columns_B.copy()

fixed_cols = list(max_column_values.keys())

print(f"📦 고정된 칼럼 {len(fixed_cols)}개 제거할 예정입니다.")

<ipython-input-5-97e9adbb6c03>:1: DtypeWarning: Columns (281,355) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('/content/drive/MyDrive/base_file/base_clean_train.csv')
<ipython-input-5-97e9adbb6c03>:2: DtypeWarning: Columns (280) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('/content/drive/MyDrive/base_file/base_clean_test.csv')


📦 고정된 칼럼 121개 제거할 예정입니다.


In [ ]:
matching_ids_train = train.copy()
for col, value in max_column_values.items():
    matching_ids_train = matching_ids_train[matching_ids_train[col] == value]

matching_ids_train_grouped = matching_ids_train.groupby('ID').filter(lambda x: len(x) == 6)
matching_ids_train_list = matching_ids_train_grouped['ID'].unique()

matching_ids_test = test.copy()
for col, value in max_column_values.items():
    matching_ids_test = matching_ids_test[matching_ids_test[col] == value]

matching_ids_test_grouped = matching_ids_test.groupby('ID').filter(lambda x: len(x) == 6)
matching_ids_test_list = matching_ids_test_grouped['ID'].unique()

train_filtered = train[train['ID'].isin(matching_ids_train_list)].drop(columns=fixed_cols)
test_filtered = test[test['ID'].isin(matching_ids_test_list)].drop(columns=fixed_cols)

print(f"🚀 최종 train 데이터 shape: {train_filtered.shape}")
print(f"🚀 최종 test 데이터 shape: {test_filtered.shape}")

train_filtered.to_csv('/content/drive/MyDrive/base_file/train_vips_B.csv', index=False)
test_filtered.to_csv('/content/drive/MyDrive/base_file/test_vips_B.csv', index=False)

🚀 최종 train 데이터 shape: (127746, 625)
🚀 최종 test 데이터 shape: (31998, 624)


In [ ]:
# 해당 코드부터 v2-8 TPU 대신 L4 GPU 사용.

In [1]:
!pip install catboost==1.2.8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 25.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train = pd.read_csv('/content/drive/MyDrive/base_file/train_vips_A.csv')
test = pd.read_csv('/content/drive/MyDrive/base_file/test_vips_A.csv')

label_encoder = LabelEncoder()
train['Segment'] = label_encoder.fit_transform(train['Segment'])

X = train.drop(columns=['Segment', 'ID'])
y = train['Segment']
X_test = test.drop(columns=['ID'])

cat_features = [col for col in X.columns if X[col].dtype == 'object']
for col in cat_features:
    X[col] = X[col].astype(str)
    X_test[col] = X_test[col].astype(str)

params = {
    'iterations': 2000,
    'learning_rate': 0.05,
    'depth': 6,
    'loss_function': 'MultiClass',
    'eval_metric': 'MultiClass',
    'verbose': 100,
    'random_seed': 42,
    'task_type': 'GPU',
    'class_weights': [20, 50, 2, 1, 1],
}

n_classes = 5
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

print(f"\n🚀 단일 Model Run 시작")

all_test_probs = np.zeros((X_test.shape[0], n_classes))

for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
    print(f"📂 Fold {fold + 1}")

    X_train_fold, X_valid_fold = X.iloc[train_idx], X.iloc[valid_idx]
    y_train_fold, y_valid_fold = y.iloc[train_idx], y.iloc[valid_idx]

    model = CatBoostClassifier(**params)
    model.fit(
        X_train_fold, y_train_fold,
        eval_set=(X_valid_fold, y_valid_fold),
        cat_features=cat_features,
        early_stopping_rounds=100,
        use_best_model=True
    )

    test_probs = model.predict_proba(X_test)
    all_test_probs += test_probs

avg_test_probs = all_test_probs / kf.get_n_splits()
prob_df = pd.DataFrame(avg_test_probs, columns=[0, 1, 2, 3, 4])
prob_df['ID'] = test['ID'].values

mean_probs = prob_df.groupby('ID').mean().reset_index()
mean_probs['Segment'] = mean_probs[[0, 1, 2, 3, 4]].idxmax(axis=1)

segment_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}
mean_probs['Segment'] = mean_probs['Segment'].map(segment_mapping)

a_ids = mean_probs.loc[mean_probs['Segment'] == 'A', 'ID'].tolist()

print(f"\n✅ A로 분류된 ID 수 = {len(a_ids)}개")
print(f"🔎 A ID: {a_ids[:50]}")


🚀 단일 Model Run 시작
📂 Fold 1
0:	learn: 1.5336602	test: 1.5334288	best: 1.5334288 (0)	total: 29.6ms	remaining: 59.1s
100:	learn: 0.6733630	test: 0.6897028	best: 0.6897028 (100)	total: 2.02s	remaining: 37.9s
200:	learn: 0.6079425	test: 0.6302349	best: 0.6302349 (200)	total: 4.01s	remaining: 35.9s
300:	learn: 0.5725195	test: 0.5980729	best: 0.5980729 (300)	total: 5.99s	remaining: 33.8s
400:	learn: 0.5478445	test: 0.5767713	best: 0.5767713 (400)	total: 7.96s	remaining: 31.7s
500:	learn: 0.5277363	test: 0.5585611	best: 0.5585611 (500)	total: 9.94s	remaining: 29.7s
600:	learn: 0.5102823	test: 0.5425113	best: 0.5425113 (600)	total: 11.9s	remaining: 27.8s
700:	learn: 0.4955293	test: 0.5290536	best: 0.5290536 (700)	total: 13.9s	remaining: 25.8s
800:	learn: 0.4829047	test: 0.5180775	best: 0.5180775 (800)	total: 15.9s	remaining: 23.8s
900:	learn: 0.4714739	test: 0.5081184	best: 0.5081184 (900)	total: 17.9s	remaining: 21.9s
1000:	learn: 0.4607617	test: 0.4985544	best: 0.4985544 (1000)	total: 19.9s	

In [10]:
train = pd.read_csv('/content/drive/MyDrive/base_file/train_vips_B.csv')
test = pd.read_csv('/content/drive/MyDrive/base_file/test_vips_B.csv')

label_encoder = LabelEncoder()
train['Segment'] = label_encoder.fit_transform(train['Segment'])

X = train.drop(columns=['Segment', 'ID'])
y = train['Segment']
X_test = test.drop(columns=['ID'])

cat_features = [col for col in X.columns if X[col].dtype == 'object']
for col in cat_features:
    X[col] = X[col].astype(str)
    X_test[col] = X_test[col].astype(str)

params = {
    'iterations': 1000,
    'learning_rate': 0.03,
    'depth': 8,
    'loss_function': 'MultiClass',
    'eval_metric': 'MultiClass',
    'verbose': 100,
    'random_seed': 42,
    'task_type': 'GPU',
    'class_weights': [10, 10, 1, 1, 1],
}

n_classes = 5
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print(f"\n🚀 단일 Model Run 시작")

all_test_probs = np.zeros((X_test.shape[0], n_classes))

for fold, (train_idx, valid_idx) in enumerate(kf.split(X, y)):
    print(f"📂 Fold {fold + 1}")

    X_train_fold, X_valid_fold = X.iloc[train_idx], X.iloc[valid_idx]
    y_train_fold, y_valid_fold = y.iloc[train_idx], y.iloc[valid_idx]

    model = CatBoostClassifier(**params)
    model.fit(
        X_train_fold, y_train_fold,
        eval_set=(X_valid_fold, y_valid_fold),
        cat_features=cat_features,
        early_stopping_rounds=100,
        use_best_model=True
    )

    test_probs = model.predict_proba(X_test)
    all_test_probs += test_probs

avg_test_probs = all_test_probs / kf.get_n_splits()
prob_df = pd.DataFrame(avg_test_probs, columns=[0, 1, 2, 3, 4])
prob_df['ID'] = test['ID'].values

mean_probs = prob_df.groupby('ID').mean().reset_index()
mean_probs['Segment'] = mean_probs[[0, 1, 2, 3, 4]].idxmax(axis=1)

segment_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}
mean_probs['Segment'] = mean_probs['Segment'].map(segment_mapping)

b_ids = mean_probs.loc[mean_probs['Segment'] == 'B', 'ID'].tolist()

print(f"\n✅ B로 분류된 ID 수 = {len(b_ids)}개")
print(f"🔎 B ID: {b_ids[:5]}")


🚀 단일 Model Run 시작
📂 Fold 1
0:	learn: 1.5622260	test: 1.5625006	best: 1.5625006 (0)	total: 39.2ms	remaining: 39.2s
100:	learn: 0.7247664	test: 0.7443114	best: 0.7443114 (100)	total: 3.77s	remaining: 33.5s
200:	learn: 0.6400983	test: 0.6730284	best: 0.6730284 (200)	total: 7.5s	remaining: 29.8s
300:	learn: 0.5942186	test: 0.6359288	best: 0.6359288 (300)	total: 11.3s	remaining: 26.2s
400:	learn: 0.5583118	test: 0.6067227	best: 0.6067227 (400)	total: 15.1s	remaining: 22.5s
500:	learn: 0.5287072	test: 0.5833666	best: 0.5833666 (500)	total: 18.9s	remaining: 18.8s
600:	learn: 0.5014977	test: 0.5608973	best: 0.5608973 (600)	total: 22.8s	remaining: 15.1s
700:	learn: 0.4787137	test: 0.5432790	best: 0.5432790 (700)	total: 26.6s	remaining: 11.4s
800:	learn: 0.4573257	test: 0.5263010	best: 0.5263010 (800)	total: 30.4s	remaining: 7.56s
900:	learn: 0.4385123	test: 0.5118517	best: 0.5118517 (900)	total: 34.3s	remaining: 3.76s
999:	learn: 0.4204232	test: 0.4973259	best: 0.4973259 (999)	total: 38s	remai

In [15]:
base_df = pd.read_csv('/content/drive/MyDrive/base_file/base_catboost_kfold.csv')

base_df.loc[base_df['ID'].isin(a_ids), 'Segment'] = 'A'
base_df.loc[base_df['ID'].isin(b_ids), 'Segment'] = 'B'

base_df.to_csv('/content/drive/MyDrive/base_file/final_catboost.csv', index=False)

print(f"✅ Segment가 'A'로 수정된 {len(a_ids)}개 ID 반영 완료")
print(f"✅ Segment가 'B'로 수정된 {len(b_ids)}개 ID 반영 완료")
print("🎯 최종 결과 저장 완료: final_catboost.csv")

✅ Segment가 'A'로 수정된 40개 ID 반영 완료
✅ Segment가 'B'로 수정된 1개 ID 반영 완료
🎯 최종 결과 저장 완료: final_catboost.csv
